In [11]:
import json
import pyodbc
from sqlalchemy import URL, create_engine
import pandas as pd
import requests


# Creating the credentials
with open("./data.json", 'r+') as file:
    data = json.load(file)
print(data, type(data))
print(data['username'])


conn = URL.create(
    "mssql+pyodbc",
    query=dict(driver="ODBC Driver 17 for SQL Server"),
    host=data['host'],
    database=data['database'],
    username=data['username'],
    password=data["password"])

engine = create_engine(conn)


{'username': 'KPI-DB-User_reporter', 'password': '3NQ7ySjru6IixLGf/zU2/', 'host': 'kpi-db-server.database.windows.net', 'port': 1433, 'database': 'kpi-db', 'query': {'driver': 'ODBC Driver 18 for SQL Server', 'TrustServerCertificate': 'yes', 'authentication': 'ActiveDirectoryIntegrated'}} <class 'dict'>
KPI-DB-User_reporter


In [10]:
dataframe = pd.read_sql("select * from ONE_Bank_Data", engine)
dictionary = dataframe.iloc[:, 3].values
dictionary

array(['122105980', '122105980', '122238200', ..., '122238200',
       '111025453', '111025453'], dtype=object)

In [13]:
banks = {}
try:
    for i in dictionary:
        url = f'https://www.routingnumbers.info/api/name.json?rn={i}'
        response = requests.get(url)
        data = json.loads(response.content)
        if data['message'] == 'OK':
            banks[i] = data['name']
        else:
            pass
except:
    pass

routing = []
bank_name = []
for i in banks:
    routing.append(i)
for i in banks.values:
    bank_name.append(i)
new_dataframe = pd.DataFrame(list(zip(bank_name, routing)), columns=[
                             'Bank', 'Routing'])

df1 = dataframe.merge(new_dataframe, on='Routing')